## Data Resize

In [ ]:
import numpy as np       
import pandas as pd       
import glob             
import os
import matplotlib.pyplot as plt 
import cv2               

from tqdm.auto import tqdm             

In [ ]:
img_size = 720

In [ ]:
image_list = glob.glob('/kaggle/input/hubmap-organ-segmentation/train_images/*.tiff')
image_list

In [ ]:
train_df = pd.read_csv('/kaggle/input/hubmap-organ-segmentation/train.csv')

In [ ]:
os.makedirs('/kaggle/working/train_image')
os.makedirs('/kaggle/working/train_mask')

In [ ]:
# train_image
for i in tqdm(range(len(image_list))):
    image = cv2.imread(image_list[i])
    name  = image_list[i].split('/')[-1].split('.')[0]
    image = cv2.resize(image, dsize = (img_size, img_size))

    save_path = f'/kaggle/working/train_image/{name}.png'
    cv2.imwrite(save_path, image)

In [ ]:
def rle_decode(rle, height, width , fill=255, dtype=np.uint8):
    s = rle.split()  
    start, length = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])] 
    start -= 1
    mask = np.zeros(height*width, dtype=dtype)    
    for i, l in zip(start, length):
        mask[i:i+l] = fill                          
    mask = mask.reshape(width,height).T     
    mask = np.ascontiguousarray(mask)         
    return mask


def rle_encode(mask):
    m = mask.T.flatten()
    m = np.concatenate([[0], m, [0]])
    run = np.where(m[1:] != m[:-1])[0] + 1

    run[1::2] -= run[::2]
    rle =  ' '.join(str(r) for r in run)
    return rle

In [ ]:
# train_mask
for i in tqdm(range(len(train_df))):
    d = train_df.iloc[i]
    name = d['id']
    rle = d['rle']
    height = d['img_height']
    width = d['img_width']

    mask = rle_decode(rle, height, width)
    mask = cv2.resize(mask, dsize=(img_size, img_size))

    save_path = f'/kaggle/working/train_mask/{name}.png'
    cv2.imwrite(save_path, mask)